# 200_YouTube_Prep


## Purpose

In this notebook we will collect our data using the Youtube API and clean that data in preperation for merging it with our movie industry data set later

## Datasets

 - Input: n/a <br>
 - Output: YoutubeTrailers.csv, trailers.pkl

In [1]:
import json#for decoding a JSON response
import pandas as pd

# Data acquisition
The following code is the what we used to scrape the data from YouTube. This code returns no more than 500 videos at a time as that is the soft limit put in place by the developers at YouTube, this is to stop the deterioration of the data you receive from the scrape as you take larger data items. 

We ran this code several times storing the data we received, which we were getting at about 400 videos per iteration of the code. After each consecutive running of the code the only change we made was to the 'publishedAfter' and 'published Before' variable found in the 'First_url' on line 32. We incremented the dates by six months at a time so as to not reach the soft limit of 500 and gather all the videos from the channel.


In [6]:
trailerData = pd.DataFrame() 

In [ ]:
#I used some of the code in the following stackoverflow question to help me with the youTube api, https://stackoverflow.com/questions/46812867/youtube-api-not-getting-all-videos-of-the-given-channel
# This code is based on the a very good example from Coding 101 which you can find here:https://www.youtube.com/watch?v=_M_wle0Iq9M
# This python 2.7.14 example shows how to retrieve with Youtube API v3 a list of uploaded Youtube videos in a channel 
#
# The code consists of two parts:
# - The first part queries the videos in a channel and stores it in a list
# - The second part queries in detail each individual video

try:
    from urllib.request import Request, urlopen
    from urllib.parse import urlencode
except ImportError:
    # Python 2 support
    from urllib2 import Request, urlopen
    from urllib import urlencode
#import urllib #importing to use its urlencode function
#import urllib2 #for making http requests
#import json #for decoding a JSON response

API_KEY = 'INSERT API KEY HERE' # This is my unique API Key which allows me to request data from the API 
ChannelIdentifier = 'UCi8e0iOVk1fEOogdfu4YgfA' # This is the unique channel identifier used to search through a sungle channel
#
# This first part will query the list of videos uploaded of a specific channel
# The identification is done through the ChannelIdentifier hwich you have defined as a variable
# The results from this first part will be stored in the list videoMetadata. This will be used in the second part of the code below.
# This code is also based on the a very good example from Coding 101 which you can find here:https://www.youtube.com/watch?v=_M_wle0Iq9M


first_part_search = 'https://www.googleapis.com/youtube/v3/search?' 
#This first_url is what we use to request the data it specifes the date, the channel and my API KEY.
first_url = first_part_search +'part=snippet&channelId='+ChannelIdentifier+'&maxResults=50&publishedBefore=2011-12-01T00:00:00.0Z&publishedAfter=2011-06-01T00:00:00.0Z&type=video&key='+API_KEY
 
videoMetadata = [] #declaring our list
url = first_url
while True:
    response = urlopen(url) #makes the call to YouTube
    videos = json.load(response) #decodes the response so we can work with it
    for video in videos['items']:
        if video['id']['kind'] == 'youtube#video':
            videoMetadata.append(video['id']['videoId']) #Appends each videoID and link to our list
#            print ("https://www.youtube.com/watch?v="+(video['id']['videoId']))
            next_page_token = videos['nextPageToken']# stores the next page token information that is returned

    try:
        next_page_token = videos['nextPageToken']
        url = first_url + '&pageToken={}'.format(next_page_token)# inserts the next page token into the url of the request so that we can get the videos on the next page of the channel
        print("have token",next_page_token)# printing to confirm we have the token
    except:
            break
trailerList = []#declaring our list
#print("Title,Upload_Date,Views,Likes,Dislikes,Comments")
for metadata in videoMetadata:
 #   print ("https://www.youtube.com/watch?v="+metadata)  # Here the videoID is printed
    SpecificVideoID = metadata
    SpecificVideoUrl = 'https://www.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id='+SpecificVideoID+'&key='+API_KEY
    response = urlopen(SpecificVideoUrl) #makes the call to a specific YouTube
    videos = json.load(response) #decodes the response so we can work with it
    videoMetadata = [] #declaring our list
    
    for video in videos['items']: # Here were checking that the inforamtion we have is in fact a youtube video.
        if video['kind'] == 'youtube#video':
            try:#if any of the data we requested about the video is missing we skip over it as we need all this data
                trailerList.append(video['snippet']['title']+"*"+video['snippet']['publishedAt']+"*"+video['statistics']['viewCount']+"*"+video['statistics']['likeCount']+"*"+video['statistics']['dislikeCount']+"*"+video['statistics']['commentCount'])
            except:
                continue

trailers = pd.DataFrame(trailerList)#stroing the scraped data that we have in our list into this temporary dataframe

In [ ]:
trailers.head()

In [ ]:
trailers.shape

In [ ]:
trailerData = trailerData.append(trailers, ignore_index = True)

In [ ]:
trailerData.shape

This is the final step in the process of creating our dataset before we go back to the API change the dates and run it again. We are storing the data the we receive into our final dataset trailer Data as we receive it.

Next you see we stored the data in a csv file while it was still in its uncleaned form, we did this for the purpose of demonstrating that steps we took to clean it.

In [ ]:
trailerData.to_csv('../../data/raw/YouTube.csv',index=False)#creating a csv of the data 

# Data cleaning 

Here we can see that all the measures we took to ensure the data was prepared for analysis.

In [2]:
trailerData  = pd.read_csv('../../data/raw/YouTube.csv',  encoding='latin-1')# loading our csv file 

In [3]:
trailerData

,0
0,Mission Impossible: Ghost Protocol BEHIND THE ...
1,Good Will Hunting (1997) Blu-Ray Release Movie...
2,Warrior (2011) Movie Trailer HD*2011-09-07T17:...
3,Being Elmo (2011) Movie Trailer - HD*2011-09-2...
4,Senna (2011) Trailer - No Fear No Limits No Eq...
5,Tomboy (2011) Movie Trailer HD*2011-09-16T19:3...
6,This Means War (2012) Trailer - HD Movie - Chr...
7,Safe House (2012) Trailer - HD Movie - Denzel ...
8,Margin Call (2011) Official HD Trailer Debut*2...
9,Hysteria (2011) Trailer - HD Movie*2011-08-17T...


With an initial look at the data we see that its stored in a single unnamed column first we will name the column then we will be able to split our data.

In [4]:
trailerData.columns = ['data']

In [5]:
trailerData.head(10)

,data
0,Mission Impossible: Ghost Protocol BEHIND THE ...
1,Good Will Hunting (1997) Blu-Ray Release Movie...
2,Warrior (2011) Movie Trailer HD*2011-09-07T17:...
3,Being Elmo (2011) Movie Trailer - HD*2011-09-2...
4,Senna (2011) Trailer - No Fear No Limits No Eq...
5,Tomboy (2011) Movie Trailer HD*2011-09-16T19:3...
6,This Means War (2012) Trailer - HD Movie - Chr...
7,Safe House (2012) Trailer - HD Movie - Denzel ...
8,Margin Call (2011) Official HD Trailer Debut*2...
9,Hysteria (2011) Trailer - HD Movie*2011-08-17T...


In [6]:
trailerData = trailerData.data.str.split('*', expand=True,)# spliting the columns based on the asterick symbol, which we inserted beween our data when gathering it 

In [7]:
trailerData.head()

,0,1,2,3,4,5,6,7,8
0,Mission Impossible: Ghost Protocol BEHIND THE ...,2011-11-29T20:20:18.000Z,1205847,4526,89,514,None,None,None
1,Good Will Hunting (1997) Blu-Ray Release Movie...,2011-09-02T21:56:32.000Z,1470725,3475,79,133,None,None,None
2,Warrior (2011) Movie Trailer HD,2011-09-07T17:45:56.000Z,4780270,11044,385,999,None,None,None
3,Being Elmo (2011) Movie Trailer - HD,2011-09-23T16:50:14.000Z,441364,4152,68,337,None,None,None
4,Senna (2011) Trailer - No Fear No Limits No Eq...,2011-07-01T19:13:52.000Z,435871,1520,30,203,None,None,None


After splitting our data into individual columns we see that we have some columns that appear to be empty as we were only scraping 6 pieces of information we know that when we split our column there must have been some extra asterisk in at least some of our data, cause extra splits.

In [8]:
trailerData.rename(columns={0:'Title' ,1:'Published',2:'Views',3:'Likes',4:'Dislikes',5:'Comments',6:'extra',7:'extra2',8:'extra3'}, inplace=True)

In [9]:
trailerData.head()

,Title,Published,Views,Likes,Dislikes,Comments,extra,extra2,extra3
0,Mission Impossible: Ghost Protocol BEHIND THE ...,2011-11-29T20:20:18.000Z,1205847,4526,89,514,None,None,None
1,Good Will Hunting (1997) Blu-Ray Release Movie...,2011-09-02T21:56:32.000Z,1470725,3475,79,133,None,None,None
2,Warrior (2011) Movie Trailer HD,2011-09-07T17:45:56.000Z,4780270,11044,385,999,None,None,None
3,Being Elmo (2011) Movie Trailer - HD,2011-09-23T16:50:14.000Z,441364,4152,68,337,None,None,None
4,Senna (2011) Trailer - No Fear No Limits No Eq...,2011-07-01T19:13:52.000Z,435871,1520,30,203,None,None,None


What we do now is look to see how many rows we have that didn't split correctly. We do this by checking how many null rows we have in these extra columns.

In [10]:
trailerData.extra.notnull().value_counts()
# creates a mask on the data returning true is there is a value in a row of the column and false otherwise, we then count all the values

False    4508
True        1
Name: extra, dtype: int64

In [11]:
trailerData.extra2.notnull().value_counts()

False    4508
True        1
Name: extra2, dtype: int64

In [12]:
trailerData.extra3.notnull().value_counts()

False    4508
True        1
Name: extra3, dtype: int64

Now we know that its a single row that's caused this issue so now we want to find that row.

In [13]:
df = trailerData[pd.notnull(trailerData.extra)]
#create a new dataframe and store the row which has a value in the extra columns.

In [14]:
df#print the dataframe

,Title,Published,Views,Likes,Dislikes,Comments,extra,extra2,extra3
3577,5 Ways Tom Hardy Will F,,,You Up (2016) HD,2016-01-05T17:24:41.000Z,133649,1817,68,133


We can now see the cause of the issue clearly in the title column there was 'F***' which through off our split.
We now know the index off the offending row and as its not a trailer we simply remove it.

In [15]:
trailerData = trailerData.drop(trailerData.index[3577])#drop the row based on the index value

We can now remove the extra columns that were created by this row as we have we don't need them since they contain no information.

In [16]:
trailerData = trailerData.drop(['extra','extra2','extra3'],axis=1)

In [17]:
trailerData.head()

,Title,Published,Views,Likes,Dislikes,Comments
0,Mission Impossible: Ghost Protocol BEHIND THE ...,2011-11-29T20:20:18.000Z,1205847,4526,89,514
1,Good Will Hunting (1997) Blu-Ray Release Movie...,2011-09-02T21:56:32.000Z,1470725,3475,79,133
2,Warrior (2011) Movie Trailer HD,2011-09-07T17:45:56.000Z,4780270,11044,385,999
3,Being Elmo (2011) Movie Trailer - HD,2011-09-23T16:50:14.000Z,441364,4152,68,337
4,Senna (2011) Trailer - No Fear No Limits No Eq...,2011-07-01T19:13:52.000Z,435871,1520,30,203


The next step is to clean out our Published column. The data that we want from this is only the year in which a video was released as we don't need the specific date and time the video was published.

In [18]:
trailerData["Published"] = trailerData["Published"].str.split("-", expand=True)[0]

In [19]:
trailerData

,Title,Published,Views,Likes,Dislikes,Comments
0,Mission Impossible: Ghost Protocol BEHIND THE ...,2011,1205847,4526,89,514
1,Good Will Hunting (1997) Blu-Ray Release Movie...,2011,1470725,3475,79,133
2,Warrior (2011) Movie Trailer HD,2011,4780270,11044,385,999
3,Being Elmo (2011) Movie Trailer - HD,2011,441364,4152,68,337
4,Senna (2011) Trailer - No Fear No Limits No Eq...,2011,435871,1520,30,203
5,Tomboy (2011) Movie Trailer HD,2011,884320,5444,168,457
6,This Means War (2012) Trailer - HD Movie - Chr...,2011,2540156,5354,240,807
7,Safe House (2012) Trailer - HD Movie - Denzel ...,2011,7586126,11180,470,2246
8,Margin Call (2011) Official HD Trailer Debut,2011,832435,1695,69,267
9,Hysteria (2011) Trailer - HD Movie,2011,1394830,2422,114,407


We can see upon examination of the data we have that there’s a lot of rows that contain information about non-trailer videos. We have videos about what’s new to Netflix, what’s coming to DVD, behind the scenes videos as well as collections of trailers that are compiled into single videos.

We had to remove these videos from the dataset, and as we have collected these videos from a single YouTube channel, the naming conventions were the same for all the videos. This enabled us to remove videos based on certain keywords.
 

In [20]:
# here we are removing all videos containing certain keywords.
trailerData = trailerData[~trailerData['Title'].str.contains('2018')]#dont want movies realised in 2018
trailerData = trailerData[~trailerData['Title'].str.contains(' Season')]#dont want tv shows
trailerData = trailerData[~trailerData['Title'].str.contains('Best New')]#doesnt name trailer/is collection of trailers
trailerData = trailerData[~trailerData['Title'].str.contains('Instant')]#these are movie reviews
trailerData = trailerData[~trailerData['Title'].str.contains('Netflix')]#dont want things from netflix
trailerData = trailerData[~trailerData['Title'].str.contains('DVD')]#dont want anything on dvd
trailerData = trailerData[~trailerData['Title'].str.contains('Now Playing')]#tells us about whats playing in local cinema
trailerData = trailerData[~trailerData['Title'].str.contains('Now In Theaters')]#Again just telling us whats in cinema
trailerData = trailerData[~trailerData['Title'].str.contains('Why Go?')]#another form of movie review 
trailerData = trailerData[~trailerData['Title'].str.contains('ALL Trailers')]# collections of several trailers 
trailerData = trailerData[~trailerData['Title'].str.contains('Behind the Scenes')]# behind the scenes videos
trailerData = trailerData[~trailerData['Title'].str.contains('BEHIND THE SCENES')]# behind the scenes videos

In [21]:
trailerData.shape

(4297, 6)

Now that we've narrowed down our videos to contain only trailers removing about 200 videos from the dataset we can look to cleaning our title strings.

The data we have contains the entire video title and the only information we want from this is the actual name of the movie. The consistent naming convention that’s used by the channel helps here again, as the videos always start off with the name of the movie. Because of this we are able to drop most of the string after we see certain keywords as we know we won’t be removing the movie title


In [22]:
#using split we can remove the characters after and including some keywords.
trailerData["Title"] = trailerData["Title"].str.split(" Official", expand=True)[0]
trailerData["Title"] = trailerData["Title"].str.split(" Theatrical Trailer", expand=True)[0]
trailerData["Title"] = trailerData["Title"].str.split(" Teaser Trailer", expand=True)[0]
trailerData["Title"] = trailerData["Title"].str.split(" International Trailer", expand=True)[0]
trailerData["Title"] = trailerData["Title"].str.split(" Trailer", expand=True)[0]
trailerData["Title"] = trailerData["Title"].str.split(" Movie", expand=True)[0]
trailerData["Title"] = trailerData["Title"].str.split(" Red Band", expand=True)[0]
trailerData["Title"] = trailerData["Title"].str.split(" Sneak Peek", expand=True)[0]
trailerData["Title"] = trailerData["Title"].str.split(" -", expand=True)[0]
trailerData["Title"] = trailerData["Title"].str.split(" HD", expand=True)[0]
trailerData["Title"] = trailerData["Title"].str.split(" Blu-Ray", expand=True)[0]
trailerData["Title"] = trailerData["Title"].str.split(" IMAX", expand=True)[0]
trailerData["Title"] = trailerData["Title"].str.split(" 3D", expand=True)[0]

trailerData = trailerData.replace(dict(Title={'\(.*\)':''}), regex=True)# in some cases the date is put in brackets after the movie, so this removes that.

We can take a look at the new title strings below and see that we now have title strings that contain only the title of the movie. This now leaves us with a prepared dataset that we can merge with out movie industry data set at a later time.

In [23]:
trailerData.head(20)

,Title,Published,Views,Likes,Dislikes,Comments
1,Good Will Hunting,2011,1470725,3475,79,133
2,Warrior,2011,4780270,11044,385,999
3,Being Elmo,2011,441364,4152,68,337
4,Senna,2011,435871,1520,30,203
5,Tomboy,2011,884320,5444,168,457
6,This Means War,2011,2540156,5354,240,807
7,Safe House,2011,7586126,11180,470,2246
8,Margin Call,2011,832435,1695,69,267
9,Hysteria,2011,1394830,2422,114,407
10,New Year's Eve,2011,1021556,2844,89,188


In [24]:
trailerData.to_pickle('../../data/processed/trailers.pkl')#save the state of the dataset to a pickle file 

In [25]:
trailerData.to_csv('../../data/processed/YouTubeTrailers.csv',index=False)# create a csv file, which ignores the index, of our cleaned Youtube trailer data. 